In [45]:
import cv2
import numpy as np
from scipy.stats import norm




def denoise(im,J,sigma,burn_in=25,it_cnt=500):
    # perform type conversions and ensure input \in {1,-1}
    noisy = im.astype(np.float32)
    noisy[noisy==255]=1
    noisy[noisy==0]=-1

    #sigma = 2 # noise level
    psi_t = {}
    psi_t['1'] = norm.pdf(noisy,1,sigma) # $\psi_t(+1)
    psi_t['-1'] = norm.pdf(noisy,-1,sigma) # $\psi_t(-1)
    avg = np.zeros(noisy.shape)
    
    sample = noisy.copy()# start from current observation
    for i in range(it_cnt):
        # direct implemntation of equation 3 
        # note that we use convolution to perform summation for performance reasons
        k = np.array([[0,1,0],[1,0,1],[0,1,0]])
        nbr_sum = cv2.filter2D(sample,-1,k) # $\sum_{s\in nbr(t)}{x_s}$
        p1 = np.exp(J*nbr_sum) * psi_t['1']
        p0 = np.exp(-J*nbr_sum) * psi_t['-1']
        prob = p1/(p1+p0)
        
        sample_mask = np.less(np.random.random(noisy.shape),prob)

        sample[sample_mask]=1
        
        if i%100==0:
            print('it cnt %d\r'%i)
        if i>=burn_in:
            avg += sample
    avg /= (it_cnt-burn_in)
    return avg
    
    


In [57]:
im = cv2.imread('3_noise.png')[:,:,0]
denoised=denoise(im,10,5,burn_in=25,it_cnt=100)
denoised[denoised>0]=255
denoised[denoised<=0]=0
print('denoise done')
while True:
    cv2.imshow('noisy',im)
    cv2.imshow('denoised',denoised)
    if cv2.waitKey(1) &0xFF==27:
        break

it cnt 0
it cnt 100
it cnt 200
it cnt 300
it cnt 400
it cnt 500
it cnt 600
it cnt 700
it cnt 800
it cnt 900
it cnt 1000
it cnt 1100
it cnt 1200
it cnt 1300
it cnt 1400
it cnt 1500
it cnt 1600
it cnt 1700
it cnt 1800
it cnt 1900
it cnt 2000
it cnt 2100
it cnt 2200
it cnt 2300
it cnt 2400
it cnt 2500
it cnt 2600
it cnt 2700
it cnt 2800
it cnt 2900
it cnt 3000
it cnt 3100
it cnt 3200
it cnt 3300
it cnt 3400
it cnt 3500
it cnt 3600
it cnt 3700
it cnt 3800
it cnt 3900
it cnt 4000
it cnt 4100
it cnt 4200
it cnt 4300
it cnt 4400
it cnt 4500
it cnt 4600
it cnt 4700
it cnt 4800
it cnt 4900
denoise done


In [29]:
sample_mask = np.less(np.random.random(noisy.shape),prob)
sample = np.zeros(noisy.shape)-1
sample[sample_mask]=1

In [31]:

cv2.imshow('im',sample*255)
cv2.waitKey(0)

1114089

array([[ 0.19947114,  0.12098536,  0.19947114, ...,  0.19947114,
         0.19947114,  0.19947114],
       [ 0.19947114,  0.19947114,  0.19947114, ...,  0.19947114,
         0.19947114,  0.19947114],
       [ 0.19947114,  0.19947114,  0.19947114, ...,  0.19947114,
         0.19947114,  0.19947114],
       ..., 
       [ 0.12098536,  0.12098536,  0.19947114, ...,  0.12098536,
         0.19947114,  0.12098536],
       [ 0.19947114,  0.12098536,  0.19947114, ...,  0.12098536,
         0.12098536,  0.19947114],
       [ 0.12098536,  0.12098536,  0.19947114, ...,  0.12098536,
         0.12098536,  0.19947114]])

In [24]:
k

array([[0, 1, 0],
       [1, 0, 1],
       [0, 1, 0]])